A continuacion definimos los archivos que nos van a servir para poder hacer los diferentes experimentos

In [ ]:
accfile = "/content/drive/MyDrive/Colab Notebooks/Proyecto/data/pickles/BK7610.pkl"
tacfile = "/content/drive/MyDrive/Colab Notebooks/Proyecto/data/clean_tac/Labels/BK7610_clean_TAClabel.csv"

accfile_test = "/content/drive/MyDrive/Colab Notebooks/Proyecto/data/pickles/BU4707.pkl"
tacfile_test = "/content/drive/MyDrive/Colab Notebooks/Proyecto/data/clean_tac/Labels/BU4707_clean_TAClabel.csv"

# Preparacion de las features y los labels

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import pickle


def mergeLabelsWithFeatures(tac_file, acc_file):
  tac = pd.read_csv(tac_file)
  ts = tac.loc[:, 'timestamp']

  infile = open(acc_file, 'rb')
  acc = pickle.load(infile)
  infile.close()
  labels = list()
  offsetTac, offsetAcc = 0, 0
  while offsetTac < len(ts) and offsetAcc < len(acc):
    while acc[offsetAcc][-1][0] < ts[offsetTac]:
      labels.append([tac.loc[offsetTac, 'y']])
      offsetAcc += 1
      if(offsetAcc >= len(acc)):
        break
    offsetTac += 1
  labels = np.array(labels)
  labels = np.ravel(labels) # Se convierte a un array de una dimension
  return labels

In [ ]:
def getData(file):
  infile = open(file, 'rb')
  acc = pickle.load(infile)
  infile.close()

  newaccTemp = list()
  newacc = list()
  for i in range(len(acc)):
    for j in range(len(acc[i])):
      newaccTemp.append(acc[i][j][1:])
    newacc.append(np.array(newaccTemp))
    newaccTemp = list()
  newacc = np.array(newacc)
  nsamples, nx, ny = newacc.shape
  newacc = newacc.reshape((nsamples, nx*ny)) # Se convierte en una matriz de dos dimensiones
  return newacc

In [ ]:
data = getData(accfile)
labels = mergeLabelsWithFeatures(tacfile, accfile)

# Datos Alternativos

Dado que el dataset esta algo desbalanceado, y dado que el ultimo 20% de las muestras poseen la misma etiqueta, entonces consideramos como alternativa trabajar con el 80% de los datos y de esta forma el dataset queda mucho mas balanceado:

In [ ]:
def getBalancedData(dataset, labels, prct):
  assert len(dataset) == len(labels), "el tamano del dataset y de los labels deben coincidir."
  n = len(labels)
  lab= labels[0:int(n*prct)]
  dat = dataset[0:int(n*prct)]
  return dat, lab

In [ ]:
data_balanced, labels_balanced = getBalancedData(data, labels, 0.8)

Por otro lado, el dataset esta "organizado" considerando la naturaleza del dataset (time series), por lo que tambien quisimos experimentar aleatorizando el dataset:

In [ ]:
import random
def getShuffledData(dataset, labels):
  package = list(zip(dataset, labels))
  random.shuffle(package)

  newSamples, newLabels = zip(*package)
  samples = np.array(newSamples)
  lbls = np.array(newLabels)

  return samples, lbls

In [ ]:
shuffled_data, shuffledLabels = getShuffledData(data, labels)
shuffled_balanced_data, shuffled_balanced_labels = getShuffledData(data_balanced, labels_balanced)

# Visualizar informacion de datos

Ahora visualizamos cada uno de los datos:

In [ ]:
def ShowDataInfo(Y, pct, data):
  class0_train = 0
  class1_train = 0

  class1_test = 0
  class0_test = 0

  train_idx = int(len(Y) * pct)

  train_label = Y[:train_idx]
  test_label = Y[train_idx:]
  
  for i in range(len(train_label)):
    if train_label[i] == 0: class0_train += 1
    else: class1_train += 1
  
  for i in range(len(test_label)):
    if test_label[i] == 0: class0_test += 1
    else: class1_test += 1
  
  print("numero de etiquetas de entrenamiento con clase 0 con datos ", data, ": ", class0_train)
  print("numero de etiquetas de entrenamiento con clase 1 con datos ", data, ": ", class1_train)
  print("numero de etiquetas de testeo con clase 0 con datos ", data, ": ", class0_test)
  print("numero de etiquetas de testeo con clase 1 con datos ", data, ": ", class1_test)

In [ ]:
ShowDataInfo(labels, 0.8, "originales")
print("******")
ShowDataInfo(labels_balanced, 0.8, "balanceados")
print("******")
ShowDataInfo(shuffledLabels, 0.8, "aleatorizados")
print("******")
ShowDataInfo(shuffled_balanced_labels, 0.8, "aleatorizados balanceados")

numero de etiquetas de entrenamiento con clase 0 con datos  originales :  11159
numero de etiquetas de entrenamiento con clase 1 con datos  originales :  13325
numero de etiquetas de testeo con clase 0 con datos  originales :  0
numero de etiquetas de testeo con clase 1 con datos  originales :  6121
******
numero de etiquetas de entrenamiento con clase 0 con datos  balanceados :  11159
numero de etiquetas de entrenamiento con clase 1 con datos  balanceados :  8428
numero de etiquetas de testeo con clase 0 con datos  balanceados :  0
numero de etiquetas de testeo con clase 1 con datos  balanceados :  4897
******
numero de etiquetas de entrenamiento con clase 0 con datos  aleatorizados :  8929
numero de etiquetas de entrenamiento con clase 1 con datos  aleatorizados :  15555
numero de etiquetas de testeo con clase 0 con datos  aleatorizados :  2230
numero de etiquetas de testeo con clase 1 con datos  aleatorizados :  3891
******
numero de etiquetas de entrenamiento con clase 0 con datos 

# Modelo de prueba

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
def classsifier(X, Y):
  train_idx = int(X.shape[0] * 0.7)
  train_data = X[:train_idx]
  train_label = Y[:train_idx]
  test_data = X[train_idx:]
  test_label = Y[train_idx:]


  cls = RandomForestClassifier(n_estimators=500)
  cls.fit(train_data, train_label)
  ypred = cls.predict(test_data)

  score = accuracy_score(test_label, ypred)
  print("Score:", score)

In [ ]:
classsifier(data, labels)

Score: 0.7312784892554808


# VSM

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import accuracy_score

def GenerateSVM(X, Y, regularization = 1, classWeight = {0:1, 1:1}, score = "accuracy_score", folds = 4, standarization = True):
  idx = 0
  kfold = StratifiedKFold(folds)
  results = pd.DataFrame()

  trainScore = []
  testScore = []
  pct_support_vectors = []    
  for trainIndex, testIndex in kfold.split(X, Y):
    Xtrain, Xtest = X[trainIndex], X[testIndex]
    Ytrain, Ytest = Y[trainIndex], Y[testIndex]

    if(standarization):
      scaler = StandardScaler()
      Xtrain = scaler.fit_transform(Xtrain)
      Xtest = scaler.fit_transform(Xtest)

    svm = SVC(C=regularization, class_weight=classWeight)
    svm.fit(Xtrain, Ytrain)

    YtrainPredict = svm.predict(Xtrain)
    YtestPredict = svm.predict(Xtest)

    if(score == "accuracy_score"):
      trainScore.append(accuracy_score(Ytrain, YtrainPredict))
      testScore.append(accuracy_score(Ytest, YtestPredict))
    elif(score == "balanced_accuracy_score"):
      trainScore.append(balanced_accuracy_score(Ytrain, YtrainPredict))
      testScore.append(balanced_accuracy_score(Ytest, YtestPredict))

    nTrain = Xtrain.shape[0]
    pct_vs = (len(svm.support_vectors_)/nTrain)*100
    pct_support_vectors.append(pct_vs)

  results.loc[idx, 'C'] = regularization
  results.loc[idx, 'rendimiento entrenamiento'] = np.mean(trainScore)
  results.loc[idx, 'rendimiento prueba'] = np.mean(testScore)
  results.loc[idx,'% de vectores de soporte'] = np.mean(pct_support_vectors)
  results.loc[0, 'Sample Weight 0'] = classWeight[0]
  results.loc[0, 'Sample Weight 1'] = classWeight[1]
  results.loc[idx, 'medida de rendimiento'] = score

  return results


Implementacion usando la funcion dada por la libreria scikit learn

In [ ]:
def SVM(X, Y, pct, C=1, classWeight = {0:1, 1:1}, score = "accuracy_score", standarization = True):
  results = pd.DataFrame()
  train_idx = int(X.shape[0] * pct)

  train_data = X[:train_idx]
  train_label = Y[:train_idx]
  test_data = X[train_idx:]
  test_label = Y[train_idx:]

  if(standarization):
      scaler = StandardScaler()
      train_data = scaler.fit_transform(train_data)
      test_data = scaler.fit_transform(test_data)

  svm = SVC(C=C, class_weight=classWeight)
  svm.fit(train_data, train_label)

  ypred = svm.predict(test_data)
  testScore = accuracy_score(test_label, ypred)

  results.loc[0, 'C'] = C
  results.loc[0, 'rendimiento prueba'] = testScore
  results.loc[0,'% de vectores de soporte'] = (len(svm.support_vectors_)/train_data.shape[0])*100
  results.loc[0, 'Sample Weight 0'] = classWeight[0]
  results.loc[0, 'Sample Weight 1'] = classWeight[1]
  results.loc[0, 'medida de rendimiento'] = score

  return results, svm

1. Modelo con parametros por defecto y datos por defecto

In [ ]:
GenerateSVM(data, labels)

,C,rendimiento entrenamiento,rendimiento prueba,% de vectores de soporte,Sample Weight 0,Sample Weight 1,medida de rendimiento
0,1.0,0.689458,0.632892,87.465666,1.0,1.0,accuracy_score


2. Modelo cambiando el class weight de la clase 0:

In [ ]:
dataframe2 = GenerateSVM(data, labels, classWeight={0:5, 1:1})

In [ ]:
display(dataframe2)

,C,rendimiento entrenamiento,rendimiento prueba,% de vectores de soporte,Sample Weight 0,Sample Weight 1,medida de rendimiento
0,1.0,0.592583,0.488004,81.131756,5.0,1.0,accuracy_score


In [ ]:
dataframe3 = GenerateSVM(data, labels, classWeight={0:1, 1:5})
display(dataframe3)

,C,rendimiento entrenamiento,rendimiento prueba,% de vectores de soporte,Sample Weight 0,Sample Weight 1,medida de rendimiento
0,1.0,0.688992,0.633413,74.484256,1.0,5.0,accuracy_score


In [ ]:
dataframe4 = GenerateSVM(data, labels, classWeight={0:1, 1:5}, regularization=10)
display(dataframe4)

,C,rendimiento entrenamiento,rendimiento prueba,% de vectores de soporte,Sample Weight 0,Sample Weight 1,medida de rendimiento
0,10.0,0.728801,0.626412,77.465616,1.0,5.0,accuracy_score


In [ ]:
dataframe5 = GenerateSVM(shuffled_data, shuffledLabels, classWeight={0:1, 1:5}, regularization=10)
display(dataframe5)

,C,rendimiento entrenamiento,rendimiento prueba,% de vectores de soporte,Sample Weight 0,Sample Weight 1,medida de rendimiento
0,10.0,0.726945,0.639631,78.01582,1.0,5.0,accuracy_score


In [ ]:
dataframe6 = GenerateSVM(shuffled_balanced_data, shuffled_balanced_labels, classWeight={0:1, 1:1}, regularization=10)
display(dataframe6)

,C,rendimiento entrenamiento,rendimiento prueba,% de vectores de soporte,Sample Weight 0,Sample Weight 1,medida de rendimiento
0,10.0,0.756573,0.606837,85.787551,1.0,1.0,accuracy_score


In [ ]:
dataframe7 = GenerateSVM(shuffled_balanced_data, shuffled_balanced_labels, classWeight={0:1, 1:5}, regularization=10)
display(dataframe7)

,C,rendimiento entrenamiento,rendimiento prueba,% de vectores de soporte,Sample Weight 0,Sample Weight 1,medida de rendimiento
0,10.0,0.661661,0.552584,83.387599,1.0,5.0,accuracy_score


In [ ]:
dataframe8 = GenerateSVM(shuffled_balanced_data, shuffled_balanced_labels, classWeight={0:1, 1:0.1}, regularization=10)
display(dataframe8)

,C,rendimiento entrenamiento,rendimiento prueba,% de vectores de soporte,Sample Weight 0,Sample Weight 1,medida de rendimiento
0,10.0,0.61959,0.51978,72.808305,1.0,0.1,accuracy_score


In [ ]:
dataframe9, s = SVM(shuffled_data, shuffledLabels, 0.8, classWeight={0:1, 1:5}, C=10)


In [ ]:
display(dataframe9)

,C,rendimiento prueba,% de vectores de soporte,Sample Weight 0,Sample Weight 1,medida de rendimiento
0,10.0,0.635357,77.899878,1.0,5.0,accuracy_score


# Analisis Discriminante Cuadratico

In [ ]:
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
def GenerateQuadraticDiscriminantAnalysis(X, Y):
  kfold = StratifiedKFold(4)
  results = pd.DataFrame()

  trainScore = []
  testScore = []

  for trainIndex, testIndex in kfold.split(X, Y):
    Xtrain, Xtest = X[trainIndex], X[testIndex]
    Ytrain, Ytest = Y[trainIndex], Y[testIndex]
    scaler = StandardScaler()
    Xtrain = scaler.fit_transform(Xtrain)
    Xtest = scaler.fit_transform(Xtest)

    QDA = QuadraticDiscriminantAnalysis()
    QDA.fit(Xtrain, Ytrain)

    YtrainPredict = QDA.predict(Xtrain)
    YtestPredict = QDA.predict(Xtest)

    trainScore.append(balanced_accuracy_score(Ytrain, YtrainPredict))
    testScore.append(balanced_accuracy_score(Ytest, YtestPredict))

  results.loc[0, 'rendimiento entrenamiento'] = np.mean(trainScore)
  results.loc[0, 'rendimiento prueba'] = np.mean(testScore)

  return results


In [ ]:
qda1 = GenerateQuadraticDiscriminantAnalysis(data, labels)
display(qda1)

,rendimiento entrenamiento,rendimiento prueba
0,0.600053,0.554642


In [ ]:
qda2 = GenerateQuadraticDiscriminantAnalysis(data_balanced, labels_balanced)
display(qda2)

,rendimiento entrenamiento,rendimiento prueba
0,0.613855,0.556294


In [ ]:
qda3 = GenerateQuadraticDiscriminantAnalysis(shuffled_data, shuffledLabels)
display(qda3)

,rendimiento entrenamiento,rendimiento prueba
0,0.594057,0.560233


In [ ]:
qda4 = GenerateQuadraticDiscriminantAnalysis(shuffled_balanced_data, shuffled_balanced_labels)
display(qda4)

,rendimiento entrenamiento,rendimiento prueba
0,0.601842,0.564979


In [ ]:
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
def QDA(X, Y, prt):
  n = len(X)
  trainX = X[0:int(n*prt)]
  trainY = Y[0:int(n*prt)]
  testX = X[int(n*prt):]
  testY = Y[int(n*prt):]

  QD = QuadraticDiscriminantAnalysis()
  QD.fit(trainX, trainY)
  ypred = QD.predict(testX)
  score = accuracy_score(testY, ypred)
  print(score)


In [ ]:
QDA(data, labels, 0.8)

0.37545787545787546


# Gradient Boosting Tree

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler

def GBT(X, Y, num_trees):
  
  results=pd.DataFrame()
  folds = 4
  skf = StratifiedKFold(n_splits=folds)
  idx= 0
  for trees in num_trees:
    trainScore = []
    testScore = []
    for train, test in skf.split(X, Y):
      trainData = X[train,:]
      trainLabel = Y[train]
      test_df = X[test,:]
      testLabel = Y[test]

      #Crear y entrenar el modelo
      clf = GradientBoostingClassifier(n_estimators=trees, learning_rate=1.0,
                                      max_depth=8, random_state=0)
      clf.fit(trainData, trainLabel)
      #Normalizamos
      scaler = StandardScaler()
      trainData = scaler.fit_transform(trainData)
      test_df = scaler.fit_transform(test_df)
      
      #Predecir
      ytrain_pred = clf.predict(trainData)
      ypred = clf.predict(test_df)

      trainScore.append(accuracy_score(trainLabel, ytrain_pred))
      testScore.append(accuracy_score(testLabel, ypred))

    results.loc[idx,'número de arboles'] = trees
    results.loc[idx,'eficiencia de entrenamiento'] = np.mean(trainScore)
    results.loc[idx,'desviacion estandar entrenamiento'] = np.std(trainScore)
    results.loc[idx,'eficiencia de prueba'] = np.mean(testScore)
    results.loc[idx,'desviacion estandar prueba'] = np.std(testScore)
    results.loc[idx,'medida de rendimiento'] = 'accuracy_score'
    idx= idx +1
    

  return results

Se ejecuta la función de Gradient Boosting Tree con árboles= 5,10,50,100




In [ ]:
arboles = [5,10,50,100]
display(GBT(data, labels, arboles))

,número de arboles,eficiencia de entrenamiento,desviacion estandar entrenamiento,eficiencia de prueba,desviacion estandar prueba,medida de rendimiento
0,5.0,0.516963,0.036283,0.488317,0.045680,accuracy_score
1,10.0,0.516908,0.041211,0.487435,0.044263,accuracy_score
2,50.0,0.535130,0.046664,0.492401,0.047269,accuracy_score
3,100.0,0.540978,0.056681,0.495701,0.047438,accuracy_score


# Redes neuronales artificiales

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler

def MLP(X,Y, num_hidden_layers, num_neurons):
  
    folds = 4
    skf = StratifiedKFold(n_splits=folds) 
    resultados = pd.DataFrame()
    idx=0
    for hidden_layers in num_hidden_layers:
        for neurons in num_neurons:
            for j, (train, test) in enumerate(skf.split(X, Y)):
                # para almacenar errores intermedios
                Error = np.zeros(folds)
                trainData = X[train,:]
                trainLabel = Y[train]
                test_df = X[test,:]
                testLabel = Y[test]
                #Normalizamos
                scaler = StandardScaler().fit(X= trainData)       
                trainData = scaler.transform(trainData)
                test_df = scaler.transform(test_df)
                
                hidden_layer_sizes = tuple(hidden_layers*[neurons])
                modelo = MLPClassifier(hidden_layer_sizes = hidden_layer_sizes, activation = 'tanh', max_iter=400, random_state = 1)
                # Entrenar el MLP
                modelo.fit(trainData,trainLabel)
                #Predecir
                Yest = modelo.predict(test_df)
                Error[j] = accuracy_score(testLabel,Yest)
                score = accuracy_score(testLabel,Yest)
                
            resultados.loc[idx,'capas ocultas'] = hidden_layers
            resultados.loc[idx,'neuronas en capas ocultas'] = neurons 
            resultados.loc[idx,'error de prueba(media)'] = np.mean(Error)
            resultados.loc[idx,'intervalo de confianza'] = np.std(Error)
            resultados.loc[idx,'rendimiento de prueba'] = score
            resultados.loc[idx,'medida de rendimiento'] = 'accuracy_score'
            idx+=1
    return (resultados)

Se ejecuta el MLP con 2 capas ocultas y 4 variaciones de neuronas en las capas ocultas


In [ ]:
display(MLP(data,labels, [1,2], [12,16,20,24]))

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (400) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


,capas ocultas,neuronas en capas ocultas,error de prueba(media),intervalo de confianza,rendimiento de prueba,medida de rendimiento
0,1.0,12.0,0.131421,0.227627,0.525683,accuracy_score
1,1.0,16.0,0.121389,0.210253,0.485557,accuracy_score
2,1.0,20.0,0.126748,0.219534,0.506993,accuracy_score
3,1.0,24.0,0.117828,0.204084,0.471311,accuracy_score
4,2.0,12.0,0.117501,0.203518,0.470004,accuracy_score
5,2.0,16.0,0.120671,0.209007,0.482682,accuracy_score
6,2.0,20.0,0.117207,0.203008,0.468828,accuracy_score
7,2.0,24.0,0.118253,0.204819,0.473010,accuracy_score


# Ventana de Parzen


In [ ]:
from sklearn.neighbors import KernelDensity
from scipy.spatial import distance

def kernel_gaussiano(x):
    
    return np.exp((-0.5)*x**2)

def ParzenWindow(x,Data,h):
    
    h = h
    Ns = Data.shape[0]
    suma = 0
    for k in range(Ns):
        u = distance.euclidean(x,Data[k,:])
        suma += kernel_gaussiano(u/h)
    return suma

def parzenClass(X_train, Y_train, X_test, h):
      
    Yest = np.zeros(X_test.shape[0])
    clases = np.unique(Y_train)
    fds_matrix = np.zeros((X_test.shape[0], len(clases)))
    
    
    for n, sample in enumerate (X_test):

      Valores = []
      Etiquetas = []
      
      
      for label in clases:
           
           
          Etiquetas.append(X_train[Y_train==clases[label]])
          probability = ParzenWindow(X_test[n],Etiquetas[label],h)/(len(Etiquetas[label]))
          fds_matrix[n,label] = probability
          Valores.append(probability)
          Yest[n] = Valores.index(max(Valores))
    
    
    return Yest, fds_matrix

def ErrorClas(Y_lest, Y):
    
    error = 1 - np.sum(Y_lest == Y)/len(Y)
    
    return error

In [ ]:
def expParzen (X, Y, hs):
    
    # estrategia de validación.
    skf = StratifiedKFold(n_splits=4)
    resultados = pd.DataFrame()
    idx = 0
    for h in hs:
        error_temp = []
        
        for train, test in skf.split(X, Y):

            Xtrain = X[train,:]
            Ytrain = Y[train]
            Xtest = X[test,:]
            Ytest = Y[test]
            #normalizamos los datos
            scaler = StandardScaler()
            scaler.fit(Xtrain)
            Xtrain = scaler.transform(Xtrain)
            Xtest = scaler.transform(Xtest)
            
            Yest, probabilidades = parzenClass(Xtrain, Ytrain, Xtest, h)
            errorTest = ErrorClas(Yest, Ytest)
            error_temp.append(errorTest)
            score = accuracy_score(Ytest,Yest)
    
        resultados.loc[idx,'ancho de ventana'] = h 
        resultados.loc[idx,'error de prueba(media)'] = np.mean(error_temp)
        resultados.loc[idx,'error de prueba(desviación estandar)'] = np.std(error_temp)
        resultados.loc[idx,'rendimiento de prueba'] = score
        idx+=1
    return (resultados)

In [ ]:
hs = [0.05, 0.1, 1, 5]
experimentos_parzen = expParzen(data,labels, hs)
experimentos_parzen

KeyboardInterrupt: ignored

#Selección de caraterísticas

In [ ]:
from sklearn.feature_selection import RFE
import time
def recursive_feature_elimination_wrapper(estimator, feature_numbers, X,Y):
    """
    Esta función es un envoltorio del objeto RFE de sklearn

    Argumentos:
        estimator(sklearn.linear_model._logistic.LogisticRegression), El estimador LR
        feature_numbers(int), El número de características a considerar
        X (numpy.array), El arreglo numpy de características
        Y (numpy.array), El vector de etiquetas

    Retorna:
        El modelo entrenado ()
        La máscara de características seleccionada, array [longitud de caracterisitcas de X]
        El rankeo de características, array [longitud de caracterisitcas de X]
        El objeto RFE entrenado sobre el set reducido de características
        El tiempo de ejecución
    """
    rfe = RFE(estimator=estimator , n_features_to_select=feature_numbers , step=1)
    tiempo_i = time.time()
    rfe.fit(X=X , y=Y )
    time_o = time.time()-tiempo_i
    feature_mask = rfe.support_
    features_rank = rfe.ranking_
    estimator = rfe.estimator_

    return rfe, feature_mask, features_rank, estimator, time_o

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import accuracy_score

def GenerateSVM(X, Y, regularization = 1, classWeight = {0:1, 1:1}, score = "accuracy_score", folds = 4, standarization = True, features=0.7):
  idx = 0
  kfold = StratifiedKFold(folds)
  results = pd.DataFrame()

  trainScore = []
  testScore = []
  pct_support_vectors = []    
  for trainIndex, testIndex in kfold.split(X, Y):
    Xtrain, Xtest = X[trainIndex], X[testIndex]
    Ytrain, Ytest = Y[trainIndex], Y[testIndex]

    if(standarization):
      scaler = StandardScaler()
      Xtrain = scaler.fit_transform(Xtrain)
      Xtest = scaler.fit_transform(Xtest)

    svm = SVC(kernel='linear', C=regularization, class_weight=classWeight)
    rfe, _, _, _, t = recursive_feature_elimination_wrapper(estimator=svm,
                                                                        feature_numbers=features ,
                                                                        X=Xtrain,
                                                                        Y=Ytrain)

    YtrainPredict = svm.predict(Xtrain)
    YtestPredict = svm.predict(Xtest)

    if(score == "accuracy_score"):
      trainScore.append(accuracy_score(Ytrain, YtrainPredict))
      testScore.append(accuracy_score(Ytest, YtestPredict))
    elif(score == "balanced_accuracy_score"):
      trainScore.append(balanced_accuracy_score(Ytrain, YtrainPredict))
      testScore.append(balanced_accuracy_score(Ytest, YtestPredict))

    nTrain = Xtrain.shape[0]
    pct_vs = (len(svm.support_vectors_)/nTrain)*100
    pct_support_vectors.append(pct_vs)

  results.loc[idx, 'C'] = regularization
  results.loc[idx, 'rendimiento entrenamiento'] = np.mean(trainScore)
  results.loc[idx, 'rendimiento prueba'] = np.mean(testScore)
  results.loc[idx,'% de vectores de soporte'] = np.mean(pct_support_vectors)
  results.loc[0, 'Sample Weight 0'] = classWeight[0]
  results.loc[0, 'Sample Weight 1'] = classWeight[1]
  results.loc[idx, 'medida de rendimiento'] = score

  return results